### Modelo de recomendación

In [73]:
# -*- coding: utf-8 -*-
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Cargar el dataset
steam_games = pd.read_csv('./datasets/steam_games.csv')

In [7]:
# Uso las columnas necesarias
df = steam_games[['genres','id','app_name']]

# Me aseguro que 'id' sea de tipo entero
df['id'] = df['id'].astype(int)

# Limpio la columna 'genres'
df['genres'] = df['genres'].apply(lambda x: str(x).replace('.', ''))

C:\Users\daant\AppData\Local\Temp\ipykernel_13208\1209921510.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)
C:\Users\daant\AppData\Local\Temp\ipykernel_13208\1209921510.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(lambda x: str(x).replace('.', ''))


In [10]:
# Voy a utilizar el Vectorizador de texto para convertir la columnas genres en vectores numéricos.
cv = CountVectorizer()
vectores = cv.fit_transform(df['genres']).toarray()

In [11]:
# Cálculo de la Similitud del Coseno entre Vectores con el fin de evaluar la similitud entre los vectores numéricos de diferentes juegos.
similitud = cosine_similarity(vectores)

In [16]:
# Generar una función para obtener recomendaciones por título
def recomendacion(juego):
    #Se busca el índice del juego en el DataFrame original (df). Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
    indice_juego = df[df["id"] == juego].index[0]
    
    #Cálculo de Similitudes: Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
    distances = similitud[indice_juego]
    
    #Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    #Salida. Devuelve la lista de títulos recomendados.
    recommended_titles = [df.iloc[i[0]]['app_name'] for i in lista_juegos]
    
    return recommended_titles

In [17]:
# Aplicar la función a la columna 'item_id' y crear una nueva columna 'Recomendaciones'
df['Recomendaciones'] = df['id'].apply(recomendacion)

C:\Users\daant\AppData\Local\Temp\ipykernel_13208\3283681334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Recomendaciones'] = df['id'].apply(recomendacion)


In [18]:
# Elimino columnas para disminuir el tamaño del archivo
df.drop(columns=['app_name', 'genres'], inplace=True)
df

C:\Users\daant\AppData\Local\Temp\ipykernel_13208\2484515945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['app_name', 'genres'], inplace=True)


,id,Recomendaciones
0,761140,"[Pixel Puzzles 2: Anime, World of Cinema - Dir..."
1,643980,"[Shadow Hunter, Immortal Empire, Immortal Empi..."
2,670290,"[Snooker-online multiplayer snooker game!, Mal..."
3,767400,"[Atomic Adam: Episode 1, Biozone, Luxor: 5th P..."
4,772540,"[Bully: Scholarship Edition, The Tomorrow War,..."
...,...,...
27829,745400,"[Foul Play, Bloody Trapland, BattleBlock Theat..."
27830,773640,"[Eets Munchies, Wildlife Park 3, Showtime!, Sm..."
27831,733530,"[Puzzler World 2, iBomber Defense Pacific, Bum..."
27832,610660,"[Try Hard Parking, Car Mechanic Simulator 2015..."


In [19]:
# Guardar el DataFrame en un archivo CSV
df.to_csv('./datasets/sisRecomendacion.csv', index=False)